In [100]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import zipfile
import requests
from io import BytesIO

In [ ]:


# GitHub 저장소의 ZIP 파일 URL
url = 'https://github.com/yumoxu/stocknet-dataset/archive/refs/heads/master.zip'

# ZIP 파일을 다운로드하여 메모리로 읽기
response = requests.get(url)
zip_file = zipfile.ZipFile(BytesIO(response.content))

# ZIP 파일 내의 'stocknet-dataset-master/price/raw/' 폴더 경로
price_folder_path = 'stocknet-dataset-master/price/raw/'

# 모든 CSV 파일을 읽고 결합
data_frames = []
for file_name in zip_file.namelist():
    if file_name.startswith(price_folder_path) and file_name.endswith('.csv'):
        # 주식 이름을 파일명에서 추출 (예: 'raw/prices/appl.csv' -> 'appl')
        stock_name = os.path.splitext(os.path.basename(file_name))[0]

        # CSV 파일을 읽어 데이터프레임으로 변환
        with zip_file.open(file_name) as file:
            df = pd.read_csv(file)
            # 주식 이름 열 추가
            df['Stock'] = stock_name
            df['Date'] = pd.to_datetime(df['Date'])
            # df.set_index('Date', inplace=True)
            data_frames.append(df)

# 모든 데이터프레임을 하나로 결합
combined_df = pd.concat(data_frames, ignore_index=False)

# 결합된 데이터프레임 확인
print(combined_df.head())


In [ ]:
stocks = combined_df['Stock'].unique()
for stock in stocks:
  combined_df['Prev Adj'] = combined_df['Adj Close'].shift(1)
  combined_df['movement'] = combined_df.apply(
    lambda row: 1 if row['Adj Close'] > row['Prev Adj'] else 0,
    axis = 1)
  combined_df['Rate'] = combined_df['Adj Close'] / combined_df['Prev Adj']-1

In [ ]:
len(combined_df)

In [ ]:
cleaned = combined_df[~((combined_df['Rate'] > -0.0055) & (combined_df['Rate'] <= 0.005))]
len(cleaned)

In [ ]:
filtered = cleaned[(cleaned['Date']>='2014-01-01') & (cleaned['Date']<='2016-01-01')]
len(filtered)

In [ ]:
# 'Date' 열을 datetime으로 변환하고 인덱스로 설정
filtered.drop(columns='Prev Adj', errors='ignore', inplace=True)
scaler = StandardScaler()
filtered[['Open', 'High', 'Low', 'Volume', 'Adj Close','Close']] = scaler.fit_transform(filtered[['Open', 'High', 'Low', 'Volume', 'Adj Close','Close']])

In [ ]:
len(filtered[filtered['movement']==1])

In [ ]:
output_dir = r'C:\Users\dhjrz\nlp\ionlab'
filtered.to_csv(os.path.join(output_dir, 'filtered_stock'), index=False)

In [90]:
filtered = pd.read_csv(r'C:\Users\dhjrz\nlp\ionlab\filtered_stock')
filtered.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Stock,movement,Rate
0,2014-01-02,-0.099005,-0.099042,-0.098989,-0.099026,-0.099058,2.716498,AAPL,0,-0.014064
1,2014-01-03,-0.099025,-0.099066,-0.099073,-0.099113,-0.099139,4.910293,AAPL,0,-0.021966
2,2014-01-06,-0.099136,-0.099115,-0.099122,-0.099092,-0.099119,5.190362,AAPL,1,0.005453
3,2014-01-07,-0.099087,-0.099121,-0.099091,-0.099120,-0.099145,3.863908,AAPL,0,-0.007151
4,2014-01-08,-0.099126,-0.099124,-0.099086,-0.099096,-0.099122,3.048033,AAPL,1,0.006333


In [91]:
import os
import pandas as pd

# 감정 분석 데이터가 위치한 디렉토리
sentiment_data_dir = r"C:\Users\dhjrz\nlp\ionlab\sentiment"

# 주가 데이터를 한 번만 불러오기 (예시 데이터 사용)
stock_data = filtered.copy()
stock_data['date'] = pd.to_datetime(stock_data['Date'])

# 감정 분석 데이터 파일명 목록 (기업별로 존재한다고 가정)
sentiment_files = os.listdir(sentiment_data_dir)

# 감정 분석 데이터를 담을 빈 데이터프레임 생성
combined_sentiment_data = pd.DataFrame()

# 반복적으로 감정 분석 데이터와 주가 데이터 병합
for sentiment_file in sentiment_files:
    # 감정 분석 파일 경로 생성
    sentiment_file_path = os.path.join(sentiment_data_dir, sentiment_file)
    
    # 감정 분석 데이터 불러오기
    sentiment_data = pd.read_csv(sentiment_file_path)
    sentiment_data['filename'] = pd.to_datetime(sentiment_data['filename'])
    
    # 파일명에서 티커 추출 (예: "AAPL_sentiment.csv"에서 "AAPL")
    ticker = sentiment_file.split('_')[0]
    
    # 해당 티커의 주가 데이터 필터링
    stock_data_ticker = stock_data[stock_data['Stock'] == ticker]
    
    # 주가 데이터를 기준으로 감정 분석 데이터 병합
    merged_data = pd.merge(stock_data_ticker, sentiment_data, left_on='date', right_on='filename', how='left')
    
    # 병합 후 데이터 확인
    print(f"Ticker: {ticker}, Merged Columns: {merged_data.columns.tolist()}")
    
    # 병합된 데이터를 추가
    combined_sentiment_data = pd.concat([combined_sentiment_data, merged_data], ignore_index=True)

# 병합 후 데이터프레임에 없는 감정 점수 열 기본값으로 채우기
combined_sentiment_data[['cleaned_tweet', 'label', 'positive', 'negative', 'neutral', 'sentiment_score']] = combined_sentiment_data[['cleaned_tweet', 'label', 'positive', 'negative', 'neutral', 'sentiment_score']].fillna({
    'cleaned_tweet': '', 'label': '', 'positive': 0.33333, 'negative': 0.33333, 'neutral': 0.33333, 'sentiment_score': 0
})

# 최종 결과 데이터프레임 출력 (또는 필요에 따라 저장)
print(combined_sentiment_data.head())


Ticker: AAPL, Merged Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Stock', 'movement', 'Rate', 'date', 'filename', 'cleaned_tweet', 'label', 'positive', 'negative', 'neutral', 'sentiment_score']
Ticker: ABBV, Merged Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Stock', 'movement', 'Rate', 'date', 'filename', 'cleaned_tweet', 'label', 'positive', 'negative', 'neutral', 'sentiment_score']
Ticker: ABB, Merged Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Stock', 'movement', 'Rate', 'date', 'filename', 'cleaned_tweet', 'label', 'positive', 'negative', 'neutral', 'sentiment_score']
Ticker: AEP, Merged Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Stock', 'movement', 'Rate', 'date', 'filename', 'cleaned_tweet', 'label', 'positive', 'negative', 'neutral', 'sentiment_score']
Ticker: AGFS, Merged Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Stock', 'moveme